In [10]:
# The code was removed by Watson Studio for sharing.

# Capstone project - IBM Data Science Specialization
# Leveraging Foursquare data to find best location to open coffee shop in London
by Misha Obolonskyi  
25 Jan 2020

## Index
1. A description of the problem and a discussion of the background.
2. A description of the data and how it will be used to solve the problem.
3. Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
4. Results section where you discuss the results.
6. Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
7. Conclusion section where you conclude the report.

## 1. Description of the problem, its background and potential stakeholders

Today many investors and cafe owners seek different opportunities to increase revenues. In retail location might be a single most important factor that ultimately decides the success of the business. With this exercice I would like to address the problem of finding the best location for potential place for a coffee bar in London.

## 2. Description of the data and how it will be used to solve the problem 

Data that I would use:
1. Foursquare data on different locations, their success, number of reviews, visitors etc
2. Public data on London about borough economics, demographics etc (https://data.london.gov.uk/)
3. Miscellaneous data sources to complement core data sets

## 3. Methodology

   ### 3.1 Loading libraries

In [11]:
# loading libraries from Lab's notebook

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans


#!conda install -c beautifulsoup4
from bs4 import BeautifulSoup
import requests
import re

!pip install rdflib
from io import BytesIO
from zipfile import ZipFile
from rdflib import Graph


print('Libraries imported.')

Libraries imported.


In [12]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library


### 3.2 Loading and preparing data for analysis

In order to get socio-economic data about London as well as names of areas, I will work with LSOA level data.
LSOA stands for Lower Layer Super Output Areas. 
For London I am getting data from London data bank -  https://data.london.gov.uk/download/lsoa-atlas/0193f884-2ccd-49c2-968e-28aa3b1c480d/lsoa-data.csv
LSOA Atlas that plots all the relevant information is here - https://londondatastore-upload.s3.amazonaws.com/instant-atlas/lsoa-atlas/atlas.html

#### 3.2.1 Loading data

In [13]:
lsoa_atlas = pd.read_csv('https://data.london.gov.uk/download/lsoa-atlas/0193f884-2ccd-49c2-968e-28aa3b1c480d/lsoa-data.csv', header=0, encoding="ISO-8859–1", low_memory=False)
lsoa_atlas.head(10)

,Lower Super Output Area,Names,Mid-year Population Estimates;All Ages;2001,Mid-year Population Estimates;All Ages;2002,Mid-year Population Estimates;All Ages;2003,Mid-year Population Estimates;All Ages;2004,Mid-year Population Estimates;All Ages;2005,Mid-year Population Estimates;All Ages;2006,Mid-year Population Estimates;All Ages;2007,Mid-year Population Estimates;All Ages;2008,Mid-year Population Estimates;All Ages;2009,Mid-year Population Estimates;All Ages;2010,Mid-year Population Estimates;All Ages;2011,Mid-year Population Estimates;All Ages;2012,Mid-year Population Estimates;All Ages;2013,Mid-year Population Estimates;Aged 0-15;2001,Mid-year Population Estimates;Aged 0-15;2002,Mid-year Population Estimates;Aged 0-15;2003,Mid-year Population Estimates;Aged 0-15;2004,Mid-year Population Estimates;Aged 0-15;2005,Mid-year Population Estimates;Aged 0-15;2006,Mid-year Population Estimates;Aged 0-15;2007,Mid-year Population Estimates;Aged 0-15;2008,Mid-year Population Estimates;Aged 0-15;2009,Mid-year Population Estimates;Aged 0-15;2010,Mid-year Population Estimates;Aged 0-15;2011,Mid-year Population Estimates;Aged 0-15;2012,Mid-year Population Estimates;Aged 0-15;2013,Mid-year Population Estimates;Aged 16-29;2001,Mid-year Population Estimates;Aged 16-29;2002,Mid-year Population Estimates;Aged 16-29;2003,Mid-year Population Estimates;Aged 16-29;2004,Mid-year Population Estimates;Aged 16-29;2005,Mid-year Population Estimates;Aged 16-29;2006,Mid-year Population Estimates;Aged 16-29;2007,Mid-year Population Estimates;Aged 16-29;2008,Mid-year Population Estimates;Aged 16-29;2009,Mid-year Population Estimates;Aged 16-29;2010,Mid-year Population Estimates;Aged 16-29;2011,Mid-year Population Estimates;Aged 16-29;2012,Mid-year Population Estimates;Aged 16-29;2013,Mid-year Population Estimates;Aged 30-44;2001,Mid-year Population Estimates;Aged 30-44;2002,Mid-year Population Estimates;Aged 30-44;2003,Mid-year Population Estimates;Aged 30-44;2004,Mid-year Population Estimates;Aged 30-44;2005,Mid-year Population Estimates;Aged 30-44;2006,Mid-year Population Estimates;Aged 30-44;2007,Mid-year Population Estimates;Aged 30-44;2008,Mid-year Population Estimates;Aged 30-44;2009,Mid-year Population Estimates;Aged 30-44;2010,Mid-year Population Estimates;Aged 30-44;2011,Mid-year Population Estimates;Aged 30-44;2012,Mid-year Population Estimates;Aged 30-44;2013,Mid-year Population Estimates;Aged 45-64;2001,Mid-year Population Estimates;Aged 45-64;2002,Mid-year Population Estimates;Aged 45-64;2003,Mid-year Population Estimates;Aged 45-64;2004,Mid-year Population Estimates;Aged 45-64;2005,Mid-year Population Estimates;Aged 45-64;2006,Mid-year Population Estimates;Aged 45-64;2007,Mid-year Population Estimates;Aged 45-64;2008,Mid-year Population Estimates;Aged 45-64;2009,Mid-year Population Estimates;Aged 45-64;2010,Mid-year Population Estimates;Aged 45-64;2011,Mid-year Population Estimates;Aged 45-64;2012,Mid-year Population Estimates;Aged 45-64;2013,Mid-year Population Estimates;Aged 65+;2001,Mid-year Population Estimates;Aged 65+;2002,Mid-year Population Estimates;Aged 65+;2003,Mid-year Population Estimates;Aged 65+;2004,Mid-year Population Estimates;Aged 65+;2005,Mid-year Population Estimates;Aged 65+;2006,Mid-year Population Estimates;Aged 65+;2007,Mid-year Population Estimates;Aged 65+;2008,Mid-year Population Estimates;Aged 65+;2009,Mid-year Population Estimates;Aged 65+;2010,Mid-year Population Estimates;Aged 65+;2011,Mid-year Population Estimates;Aged 65+;2012,Mid-year Population Estimates;Aged 65+;2013,Mid-year Population Estimates;Working-age;2001,Mid-year Population Estimates;Working-age;2002,Mid-year Population Estimates;Working-age;2003,Mid-year Population Estimates;Working-age;2004,Mid-year Population Estimates;Working-age;2005,Mid-year Population Estimates;Working-age;2006,Mid-year Population Estimates;Working-age;2007,Mid-year Population Estimates;Working-age;2008,Mid-year Population Estimates;Working-age;2009,Mid-year Population Est

In [14]:
lsoa_atlas.shape

(4837, 277)

Based on head and shape we see that file contains vast amoung of information for each LSOA. For further analysis, I'll focus only on LSOAs in central London and key socio-economic data

#### 3.2.2 Data cleaning and transformation

As regards LSOA, I'll narrow analysis to areas in Central London that organized in boroughs:
- City of London
- Westminster
- Camden
- Islington
- Tower Hamlets
- Hackney
- Kensington and Chelsea

In [15]:
lsoa_london_names = ['City of London', 'Westminster', 'Kensington and Chelsea']
lsoa_london = lsoa_atlas[lsoa_atlas.Names.str.contains('|'.join(lsoa_london_names), na=False)].reset_index()
lsoa_london.head()

,index,Lower Super Output Area,Names,Mid-year Population Estimates;All Ages;2001,Mid-year Population Estimates;All Ages;2002,Mid-year Population Estimates;All Ages;2003,Mid-year Population Estimates;All Ages;2004,Mid-year Population Estimates;All Ages;2005,Mid-year Population Estimates;All Ages;2006,Mid-year Population Estimates;All Ages;2007,Mid-year Population Estimates;All Ages;2008,Mid-year Population Estimates;All Ages;2009,Mid-year Population Estimates;All Ages;2010,Mid-year Population Estimates;All Ages;2011,Mid-year Population Estimates;All Ages;2012,Mid-year Population Estimates;All Ages;2013,Mid-year Population Estimates;Aged 0-15;2001,Mid-year Population Estimates;Aged 0-15;2002,Mid-year Population Estimates;Aged 0-15;2003,Mid-year Population Estimates;Aged 0-15;2004,Mid-year Population Estimates;Aged 0-15;2005,Mid-year Population Estimates;Aged 0-15;2006,Mid-year Population Estimates;Aged 0-15;2007,Mid-year Population Estimates;Aged 0-15;2008,Mid-year Population Estimates;Aged 0-15;2009,Mid-year Population Estimates;Aged 0-15;2010,Mid-year Population Estimates;Aged 0-15;2011,Mid-year Population Estimates;Aged 0-15;2012,Mid-year Population Estimates;Aged 0-15;2013,Mid-year Population Estimates;Aged 16-29;2001,Mid-year Population Estimates;Aged 16-29;2002,Mid-year Population Estimates;Aged 16-29;2003,Mid-year Population Estimates;Aged 16-29;2004,Mid-year Population Estimates;Aged 16-29;2005,Mid-year Population Estimates;Aged 16-29;2006,Mid-year Population Estimates;Aged 16-29;2007,Mid-year Population Estimates;Aged 16-29;2008,Mid-year Population Estimates;Aged 16-29;2009,Mid-year Population Estimates;Aged 16-29;2010,Mid-year Population Estimates;Aged 16-29;2011,Mid-year Population Estimates;Aged 16-29;2012,Mid-year Population Estimates;Aged 16-29;2013,Mid-year Population Estimates;Aged 30-44;2001,Mid-year Population Estimates;Aged 30-44;2002,Mid-year Population Estimates;Aged 30-44;2003,Mid-year Population Estimates;Aged 30-44;2004,Mid-year Population Estimates;Aged 30-44;2005,Mid-year Population Estimates;Aged 30-44;2006,Mid-year Population Estimates;Aged 30-44;2007,Mid-year Population Estimates;Aged 30-44;2008,Mid-year Population Estimates;Aged 30-44;2009,Mid-year Population Estimates;Aged 30-44;2010,Mid-year Population Estimates;Aged 30-44;2011,Mid-year Population Estimates;Aged 30-44;2012,Mid-year Population Estimates;Aged 30-44;2013,Mid-year Population Estimates;Aged 45-64;2001,Mid-year Population Estimates;Aged 45-64;2002,Mid-year Population Estimates;Aged 45-64;2003,Mid-year Population Estimates;Aged 45-64;2004,Mid-year Population Estimates;Aged 45-64;2005,Mid-year Population Estimates;Aged 45-64;2006,Mid-year Population Estimates;Aged 45-64;2007,Mid-year Population Estimates;Aged 45-64;2008,Mid-year Population Estimates;Aged 45-64;2009,Mid-year Population Estimates;Aged 45-64;2010,Mid-year Population Estimates;Aged 45-64;2011,Mid-year Population Estimates;Aged 45-64;2012,Mid-year Population Estimates;Aged 45-64;2013,Mid-year Population Estimates;Aged 65+;2001,Mid-year Population Estimates;Aged 65+;2002,Mid-year Population Estimates;Aged 65+;2003,Mid-year Population Estimates;Aged 65+;2004,Mid-year Population Estimates;Aged 65+;2005,Mid-year Population Estimates;Aged 65+;2006,Mid-year Population Estimates;Aged 65+;2007,Mid-year Population Estimates;Aged 65+;2008,Mid-year Population Estimates;Aged 65+;2009,Mid-year Population Estimates;Aged 65+;2010,Mid-year Population Estimates;Aged 65+;2011,Mid-year Population Estimates;Aged 65+;2012,Mid-year Population Estimates;Aged 65+;2013,Mid-year Population Estimates;Working-age;2001,Mid-year Population Estimates;Working-age;2002,Mid-year Population Estimates;Working-age;2003,Mid-year Population Estimates;Working-age;2004,Mid-year Population Estimates;Working-age;2005,Mid-year Population Estimates;Working-age;2006,Mid-year Population Estimates;Working-age;2007,Mid-year Population Estimates;Working-age;2008,Mid-year Population Estimates;Working-age;2009,Mid-year Populati

In [16]:
#'Camden', 'Islington', 'Tower Hamlets', 'Hackney',

In [17]:
lsoa_london.shape

(237, 278)

Data I'll keep for further analysis:
- Population Density;Persons per hectare;2013
- Households;All households;2011
- House Prices;Median Price (£);2009
- House Prices;Median Price (£);2014
- House Prices;Sales;2009
- House Prices;Sales;2014
- Economic Activity;Economically active: Total;2011
- Economic Activity;Unemployment Rate;2011
- Household Income, 2011/12;Mean Annual Household Income estimate (£)

In [18]:
lsoa_london = lsoa_london[['Lower Super Output Area',
                           'Names',
                           'Population Density;Persons per hectare;2013',
                           'Households;All households;2011',
                           'House Prices;Median Price (£);2009',
                           'House Prices;Median Price (£);2014',
                           'House Prices;Sales;2009',
                           'House Prices;Sales;2014',
                           'Economic Activity;Economically active: Total;2011',
                           'Economic Activity;Unemployment Rate;2011',
                           'Household Income, 2011/12;Mean Annual Household Income estimate (£)']]
lsoa_london.head()

,Lower Super Output Area,Names,Population Density;Persons per hectare;2013,Households;All households;2011,House Prices;Median Price (£);2009,House Prices;Median Price (£);2014,House Prices;Sales;2009,House Prices;Sales;2014,Economic Activity;Economically active: Total;2011,Economic Activity;Unemployment Rate;2011,"Household Income, 2011/12;Mean Annual Household Income estimate (£)"
0,E01000001,City of London 001A,114.0,876.0,480000,720000,41.0,67.0,964.0,3.5,74846.0
1,E01000002,City of London 001B,64.0,830.0,491750,836500,28.0,66.0,926.0,1.7,73325.0
2,E01000003,City of London 001C,230.0,817.0,349000,487500,24.0,30.0,822.0,4.7,44130.0
3,E01000005,City of London 001E,62.0,467.0,195000,414500,4.0,6.0,530.0,8.7,30781.0
4,E01032739,City of London 001F,6.0,676.0,340000,757125,39.0,70.0,843.0,2.7,68022.0


Also checking for types of data in the dataframe and converting them in proper datatypes

In [19]:
lsoa_london.dtypes

Lower Super Output Area                                                 object
Names                                                                   object
Population Density;Persons per hectare;2013                            float64
Households;All households;2011                                         float64
House Prices;Median Price (£);2009                                      object
House Prices;Median Price (£);2014                                      object
House Prices;Sales;2009                                                float64
House Prices;Sales;2014                                                float64
Economic Activity;Economically active: Total;2011                      float64
Economic Activity;Unemployment Rate;2011                               float64
Household Income, 2011/12;Mean Annual Household Income estimate (£)    float64
dtype: object

In [20]:
lsoa_london['House Prices;Median Price (£);2009'] = pd.to_numeric(lsoa_london['House Prices;Median Price (£);2009'], errors='coerce')
lsoa_london['House Prices;Median Price (£);2014'] = pd.to_numeric(lsoa_london['House Prices;Median Price (£);2014'], errors='coerce')

In [21]:
lsoa_london.dtypes

Lower Super Output Area                                                 object
Names                                                                   object
Population Density;Persons per hectare;2013                            float64
Households;All households;2011                                         float64
House Prices;Median Price (£);2009                                     float64
House Prices;Median Price (£);2014                                       int64
House Prices;Sales;2009                                                float64
House Prices;Sales;2014                                                float64
Economic Activity;Economically active: Total;2011                      float64
Economic Activity;Unemployment Rate;2011                               float64
Household Income, 2011/12;Mean Annual Household Income estimate (£)    float64
dtype: object

Once all datatypes are in the appropriate format, I can calculate CAGRs for House median Prices and House Sales

In [22]:
lsoa_london['House Prices;Median Price (£);CAGR'] = ((lsoa_london['House Prices;Median Price (£);2014'] / lsoa_london['House Prices;Median Price (£);2009']) ** (1/6)) - 1 
lsoa_london['House Prices;Sales;CAGR'] = ((lsoa_london['House Prices;Sales;2014'] / lsoa_london['House Prices;Sales;2009']) ** (1/6)) - 1 

In [23]:
lsoa_london.head(10)

,Lower Super Output Area,Names,Population Density;Persons per hectare;2013,Households;All households;2011,House Prices;Median Price (£);2009,House Prices;Median Price (£);2014,House Prices;Sales;2009,House Prices;Sales;2014,Economic Activity;Economically active: Total;2011,Economic Activity;Unemployment Rate;2011,"Household Income, 2011/12;Mean Annual Household Income estimate (£)",House Prices;Median Price (£);CAGR,House Prices;Sales;CAGR
0,E01000001,City of London 001A,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297
1,E01000002,City of London 001B,64.0,830.0,491750.0,836500,28.0,66.0,926.0,1.7,73325.0,0.092581,0.153624
2,E01000003,City of London 001C,230.0,817.0,349000.0,487500,24.0,30.0,822.0,4.7,44130.0,0.057284,0.037891
3,E01000005,City of London 001E,62.0,467.0,195000.0,414500,4.0,6.0,530.0,8.7,30781.0,0.133918,0.069913
4,E01032739,City of London 001F,6.0,676.0,340000.0,757125,39.0,70.0,843.0,2.7,68022.0,0.142742,0.102399
5,E01032740,City of London 001G,18.0,719.0,320000.0,850800,21.0,70.0,887.0,3.3,67266.0,0.177008,0.222212
6,E01002852,Kensington and Chelsea 001A,39.0,861.0,224799.0,336375,12.0,8.0,943.0,12.0,33341.0,0.069477,-0.065345
7,E01002853,Kensington and Chelsea 001B,162.0,470.0,340000.0,910000,3.0,17.0,493.0,13.4,33794.0,0.178312,0.335225
8,E01002854,Kensington and Chelsea 001C,90.0,488.0,NaN,372000,0.0,5.0,509.0,15.7,33010.0,NaN,inf
9,E01002855,Kensington and Chelsea 001D,157.0,764.0,342250.0,469250,8.0,16.0,845.0,13.3,36094.0,0.054007,0.122462


In [24]:
lsoa_london.shape

(237, 13)

#### 3.2.3 Enriching LSOA London data with coordinates

https://opendatacommunities.org/downloads/graph?uri=http://opendatacommunities.org/graph/lower-layer-super-output-areas

In [25]:
url = 'https://opendatacommunities-downloads.s3.amazonaws.com/lsoa.ttl.zip'
content = requests.get(url)

g = Graph()

f = ZipFile(BytesIO(content.content))
print(f.namelist())

g.parse(f.open('lsoa.ttl'), format='turtle')


['lsoa.ttl']


<Graph identifier=N659730f3c7914430a4e4e66374c5a01e (<class 'rdflib.graph.Graph'>)>

In [26]:
len(g)

273128

In [27]:
qres = g.query(
    """select ?nlsoa ?label ?lat ?long
        where {
            ?nlsoa rdfs:label ?label;
            geo:lat ?lat;
            geo:long ?long.
            }""")

res = [{str(k): str(v) for k, v in binding.items()} for binding in qres.bindings]

In [28]:
df = pd.DataFrame(res)

In [29]:
df.head(10)

,label,lat,long,nlsoa
0,Poole 010E,50.73191,-1.91827,http://opendatacommunities.org/id/geography/ls...
1,South Gloucestershire 029A,51.45423,-2.48253,http://opendatacommunities.org/id/geography/ls...
2,Colchester 014D,51.87661,0.86079,http://opendatacommunities.org/id/geography/ls...
3,Rushmoor 006C,51.28911,-0.76205,http://opendatacommunities.org/id/geography/ls...
4,"Rhondda, Cynon, Taff 026B",51.58367,-3.29986,http://opendatacommunities.org/id/geography/ls...
5,Epsom and Ewell 002A,51.36363,-0.26570,http://opendatacommunities.org/id/geography/ls...
6,Bradford 041C,53.79558,-1.77918,http://opendatacommunities.org/id/geography/ls...
7,Cherwell 016E,51.82977,-1.16277,http://opendatacommunities.org/id/geography/ls...
8,Sunderland 031C,54.86049,-1.41740,http://opendatacommunities.org/id/geography/ls...
9,Wrexham 006D,53.05676,-3.06988,http://opendatacommunities.org/id/geography/ls...


In [30]:
df.nlsoa = df.nlsoa.str.strip("http://opendatacommunities.org/id/geography/lsoa/")

In [31]:
df.head()

,label,lat,long,nlsoa
0,Poole 010E,50.73191,-1.91827,E01015394
1,South Gloucestershire 029A,51.45423,-2.48253,E01014943
2,Colchester 014D,51.87661,0.86079,E01021692
3,Rushmoor 006C,51.28911,-0.76205,E01023095
4,"Rhondda, Cynon, Taff 026B",51.58367,-3.29986,W01001239


In [32]:
df.shape

(34378, 4)

In [33]:
df.rename(columns={'label':'LSOA name', 
                  'lat':'Latitude',
                  'long':'Longitude',
                  'nlsoa':'Lower Super Output Area'},
         inplace=True)

In [34]:
lsoa_london_coordinates = pd.merge(lsoa_london, df, how='left', on='Lower Super Output Area')

In [35]:
lsoa_london_coordinates.head()

,Lower Super Output Area,Names,Population Density;Persons per hectare;2013,Households;All households;2011,House Prices;Median Price (£);2009,House Prices;Median Price (£);2014,House Prices;Sales;2009,House Prices;Sales;2014,Economic Activity;Economically active: Total;2011,Economic Activity;Unemployment Rate;2011,"Household Income, 2011/12;Mean Annual Household Income estimate (£)",House Prices;Median Price (£);CAGR,House Prices;Sales;CAGR,LSOA name,Latitude,Longitude
0,E01000001,City of London 001A,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297,City of London 001A,51.51801,-0.09677
1,E01000002,City of London 001B,64.0,830.0,491750.0,836500,28.0,66.0,926.0,1.7,73325.0,0.092581,0.153624,City of London 001B,51.51818,-0.09255
2,E01000003,City of London 001C,230.0,817.0,349000.0,487500,24.0,30.0,822.0,4.7,44130.0,0.057284,0.037891,City of London 001C,51.52171,-0.09582
3,E01000005,City of London 001E,62.0,467.0,195000.0,414500,4.0,6.0,530.0,8.7,30781.0,0.133918,0.069913,City of London 001E,51.51349,-0.07579
4,E01032739,City of London 001F,6.0,676.0,340000.0,757125,39.0,70.0,843.0,2.7,68022.0,0.142742,0.102399,NaN,NaN,NaN


In [36]:
lsoa_london_coordinates = lsoa_london_coordinates.drop(['LSOA name'], axis=1)

In [37]:
lsoa_london_coordinates.head(10)

,Lower Super Output Area,Names,Population Density;Persons per hectare;2013,Households;All households;2011,House Prices;Median Price (£);2009,House Prices;Median Price (£);2014,House Prices;Sales;2009,House Prices;Sales;2014,Economic Activity;Economically active: Total;2011,Economic Activity;Unemployment Rate;2011,"Household Income, 2011/12;Mean Annual Household Income estimate (£)",House Prices;Median Price (£);CAGR,House Prices;Sales;CAGR,Latitude,Longitude
0,E01000001,City of London 001A,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297,51.51801,-0.09677
1,E01000002,City of London 001B,64.0,830.0,491750.0,836500,28.0,66.0,926.0,1.7,73325.0,0.092581,0.153624,51.51818,-0.09255
2,E01000003,City of London 001C,230.0,817.0,349000.0,487500,24.0,30.0,822.0,4.7,44130.0,0.057284,0.037891,51.52171,-0.09582
3,E01000005,City of London 001E,62.0,467.0,195000.0,414500,4.0,6.0,530.0,8.7,30781.0,0.133918,0.069913,51.51349,-0.07579
4,E01032739,City of London 001F,6.0,676.0,340000.0,757125,39.0,70.0,843.0,2.7,68022.0,0.142742,0.102399,NaN,NaN
5,E01032740,City of London 001G,18.0,719.0,320000.0,850800,21.0,70.0,887.0,3.3,67266.0,0.177008,0.222212,NaN,NaN
6,E01002852,Kensington and Chelsea 001A,39.0,861.0,224799.0,336375,12.0,8.0,943.0,12.0,33341.0,0.069477,-0.065345,51.52685,-0.21904
7,E01002853,Kensington and Chelsea 001B,162.0,470.0,340000.0,910000,3.0,17.0,493.0,13.4,33794.0,0.178312,0.335225,51.52268,-0.21093
8,E01002854,Kensington and Chelsea 001C,90.0,488.0,NaN,372000,0.0,5.0,509.0,15.7,33010.0,NaN,inf,51.52247,-0.20460
9,E01002855,Kensington and Chelsea 001D,157.0,764.0,342250.0,469250,8.0,16.0,845.0,13.3,36094.0,0.054007,0.122462,51.52070,-0.20750


In [38]:
lsoa_london_coordinates.tail(10)

,Lower Super Output Area,Names,Population Density;Persons per hectare;2013,Households;All households;2011,House Prices;Median Price (£);2009,House Prices;Median Price (£);2014,House Prices;Sales;2009,House Prices;Sales;2014,Economic Activity;Economically active: Total;2011,Economic Activity;Unemployment Rate;2011,"Household Income, 2011/12;Mean Annual Household Income estimate (£)",House Prices;Median Price (£);CAGR,House Prices;Sales;CAGR,Latitude,Longitude
227,E01004668,Westminster 023D,272.0,645.0,800000.0,565000,11.0,15.0,725.0,6.9,55516.0,-0.056316,0.053052,51.48707,-0.14451
228,E01004750,Westminster 023E,73.0,951.0,537500.0,1098000,32.0,39.0,973.0,5.5,59386.0,0.126429,0.033521,51.49365,-0.14609
229,E01033597,Westminster 023F,189.0,621.0,660000.0,995000,19.0,21.0,591.0,7.4,55477.0,0.070812,0.016820,NaN,NaN
230,E01033599,Westminster 023G,283.0,630.0,140600.0,570000,15.0,5.0,672.0,10.4,40838.0,0.262743,-0.167317,NaN,NaN
231,E01004665,Westminster 024A,321.0,611.0,215000.0,477500,3.0,8.0,559.0,10.4,32327.0,0.142236,0.177592,51.48635,-0.14166
232,E01004669,Westminster 024B,233.0,758.0,272500.0,418070,10.0,14.0,772.0,9.5,42416.0,0.073941,0.057681,51.48526,-0.14598
233,E01004737,Westminster 024C,177.0,683.0,699000.0,1367500,9.0,18.0,687.0,7.4,69828.0,0.118343,0.122462,51.48789,-0.13100
234,E01004738,Westminster 024D,158.0,722.0,625000.0,861956,27.0,28.0,884.0,4.4,90092.0,0.055037,0.006080,51.48614,-0.13434
235,E01004739,Westminster 024E,479.0,950.0,450000.0,580000,3.0,1.0,929.0,2.6,80463.0,0.043204,-0.167317,51.48641,-0.13640
236,E01004741,Westminster 024F,236.0,698.0,395000.0,579000,13.0,19.0,766.0,5.0,56027.0,0.065811,0.065291,51.48937,-0.13423


In [39]:
lsoa_london_coordinates.isnull().sum()

Lower Super Output Area                                                 0
Names                                                                   0
Population Density;Persons per hectare;2013                             0
Households;All households;2011                                          0
House Prices;Median Price (£);2009                                      2
House Prices;Median Price (£);2014                                      0
House Prices;Sales;2009                                                 0
House Prices;Sales;2014                                                 0
Economic Activity;Economically active: Total;2011                       0
Economic Activity;Unemployment Rate;2011                                0
Household Income, 2011/12;Mean Annual Household Income estimate (£)     0
House Prices;Median Price (£);CAGR                                      2
House Prices;Sales;CAGR                                                 0
Latitude                              

Dropping rows with NaN values

In [40]:
lsoa_london_coordinates_clean = lsoa_london_coordinates.dropna()

In [41]:
#project.save_data(file_name = "lsoa_data.csv", data = lsoa_london_coordinates_clean.to_csv(index=False),overwrite=True)

{'file_name': 'lsoa_data.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstoneibmdatascience-donotdelete-pr-ur8dfm3zrk3isl',
 'asset_id': '92b062fb-3421-410e-bf92-6aea2feab23d'}

#### 3.2.4 Maping London LSOAs

In [42]:
address = 'London, UK'

geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [43]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, names in zip(lsoa_london_coordinates_clean['Latitude'], lsoa_london_coordinates_clean['Longitude'], lsoa_london_coordinates_clean['Names']):
    label = '{}'.format(names)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

### 3.3 Exporting data from Foursquare

In [44]:
CLIENT_ID = 'JFNXSGMCLJZDGSZLXRZ1XQY4XEWTXENCMKGI3Y52J1MY3PLE' # your Foursquare ID
CLIENT_SECRET = '44WBAMLF1OTWP5XLQXCS3O1A5BKUFNKCVRM3IXDP140XM3KY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=30):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL      
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['LSOA name', 'LSOA Latitude', 'LSOA Longitude', 'Venue ID', 'Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [46]:
#london_f_test = lsoa_london_coordinates_clean[:5]

In [47]:
london_venues = getNearbyVenues(names=lsoa_london_coordinates_clean['Names'],
                                   latitudes=lsoa_london_coordinates_clean['Latitude'],
                                   longitudes=lsoa_london_coordinates_clean['Longitude']
                                  )

City of London 001A
City of London 001B
City of London 001C
City of London 001E
Kensington and Chelsea 001A
Kensington and Chelsea 001B
Kensington and Chelsea 001D
Kensington and Chelsea 001E
Kensington and Chelsea 002A
Kensington and Chelsea 002B
Kensington and Chelsea 002C
Kensington and Chelsea 002D
Kensington and Chelsea 003A
Kensington and Chelsea 003B
Kensington and Chelsea 003C
Kensington and Chelsea 003D
Kensington and Chelsea 003E
Kensington and Chelsea 004A
Kensington and Chelsea 004B
Kensington and Chelsea 004C
Kensington and Chelsea 004D
Kensington and Chelsea 004E
Kensington and Chelsea 005A
Kensington and Chelsea 005B
Kensington and Chelsea 005C
Kensington and Chelsea 005D
Kensington and Chelsea 005E
Kensington and Chelsea 006A
Kensington and Chelsea 006B
Kensington and Chelsea 006C
Kensington and Chelsea 006D
Kensington and Chelsea 006E
Kensington and Chelsea 007A
Kensington and Chelsea 007B
Kensington and Chelsea 007C
Kensington and Chelsea 007D
Kensington and Chelsea 0

In [48]:
london_venues.head()

,LSOA name,LSOA Latitude,LSOA Longitude,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,City of London 001A,51.51801,-0.09677,4fc31eede4b05b8503be268b,Virgin Active,51.517952,-0.097651,Gym / Fitness Center
1,City of London 001A,51.51801,-0.09677,4ad3be62f964a52012e620e3,Postman's Park,51.516860,-0.097643,Park
2,City of London 001A,51.51801,-0.09677,4ac518d2f964a5203ca720e3,Museum of London,51.518019,-0.096060,History Museum
3,City of London 001A,51.51801,-0.09677,4ad4f4c0f964a520e70021e3,St Bartholomew the Great (St Bartholomew-the-G...,51.518631,-0.099890,Church
4,City of London 001A,51.51801,-0.09677,4c1cd32bb306c928426b64b7,Barbican Art Gallery,51.519800,-0.093969,Art Gallery


In [49]:
london_venues.shape

(5947, 8)

Based on the scrapping of all venues we see that number of places is quite significant. For the purpose of analysis we need to narrow the list of eating places such as Cafe, Restaurant, Bar, Pub etc. 
List of venue categories - https://developer.foursquare.com/docs/resources/categories

In [50]:
eating_names = ['Cafe', 'Coffee', 'Pub', 'Restaurant', 'Bar', 'Cha Chaan Teng', 'Breakfast', 'BBQ Joint', 'Bagel', 'Bakery', 'Bistro', 'Tea', 'Buffet', 'Burger Joint', 'Cafeteria', 'Creperie', 'Deli', 'Dessert', 'Diner', 'Donut','Food Court', 'Friterie', 'Gastropub', 'Pizza', 'Salad', 'Steakhouse', 'Brewery', 'Lounge']
london_eats = london_venues[london_venues['Venue Category'].str.contains('|'.join(eating_names), na=False)].reset_index()

In [51]:
london_eats.shape

(3107, 9)

In [52]:
london_eats = london_eats[:300]

In [53]:
london_eats.head()

,index,LSOA name,LSOA Latitude,LSOA Longitude,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,9,City of London 001A,51.51801,-0.09677,4ad7a8ddf964a520650d21e3,Dose Espresso,51.519553,-0.099406,Coffee Shop
1,11,City of London 001A,51.51801,-0.09677,5384e5ed498e11317d174b6d,Ask For Janice,51.519128,-0.100374,Modern European Restaurant
2,12,City of London 001A,51.51801,-0.09677,4ada5cf0f964a520e32121e3,The Old Red Cow,51.519607,-0.099157,Beer Bar
3,13,City of London 001A,51.51801,-0.09677,4dff32bae4cdf7246077a9aa,Pilpel,51.515195,-0.098462,Falafel Restaurant
4,18,City of London 001A,51.51801,-0.09677,4ac518d6f964a5201ea820e3,Club Gascon,51.518541,-0.100462,French Restaurant


In [54]:
london_eats.shape

(300, 9)

In [54]:
project.save_data(file_name = "london_eats.csv", data = london_eats.to_csv(index=False),overwrite=True)

{'file_name': 'london_eats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstoneibmdatascience-donotdelete-pr-ur8dfm3zrk3isl',
 'asset_id': 'e6767020-a6fd-46d5-8e77-885bf50e21c8'}

In [44]:
def getVenuesStats(venue_id, LIMIT=100):
    
    venues_stats=[]
    
    for id_venue in venue_id:
                    
        # create the API request URL      
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&limit={}'.format(
            id_venue,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['venue'] 
      
        if ('rating') in results:
            rating = results['rating']
        else: rating = 'NaN'
            
        if ('stats' in results):
            tipCount = results['stats']['tipCount']
        else: tipCount = 'NaN'

        
        if ('price') in results:
            price = results['price']['tier']
        else: price='NaN'
                  
        # return only relevant information for venue
        
        venues_stats.append([
                id_venue,  
                results['name'],
                tipCount,
                rating,
                price])
        
        
    nearby_venues_stats = pd.DataFrame(venues_stats)
    nearby_venues_stats.columns = ['Venue ID',
                                   'Venue name',
                                   'Tip Count',
                                   'Rating',
                                   'Price tier']
    
    return(nearby_venues_stats)

Considering recent changes for Foursquare API, it is possible to get only number of tips for venue. Although number of users and checkins would show better picture about trendiness of the place, I believe that number of tips is also a good proxy of that.  

In [45]:
london_venues_stats = getVenuesStats(venue_id=london_eats['Venue ID'])

In [46]:
london_venues_stats.head()
london_venues_stats.shape

,Venue ID,Venue name,Tip Count,Rating,Price tier
0,4ad7a8ddf964a520650d21e3,Dose Espresso,69,8.4,2
1,5384e5ed498e11317d174b6d,Ask For Janice,65,8.4,2
2,4ada5cf0f964a520e32121e3,The Old Red Cow,59,8.2,2
3,4dff32bae4cdf7246077a9aa,Pilpel,36,9.1,2
4,4ac518d6f964a5201ea820e3,Club Gascon,21,8,3


In [71]:
#project.save_data(file_name = "foursquare_stats.csv", data = london_venues_stats.to_csv(index=False),overwrite=True)

{'file_name': 'foursquare_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstoneibmdatascience-donotdelete-pr-ur8dfm3zrk3isl',
 'asset_id': '30c95f6f-04d1-47a5-b140-46770d77cb3d'}

### 3.4 Data transformation

In [5]:
my_file = project.get_file("foursquare_stats.csv")
df = pd.read_csv(my_file)
london_venues_stats = df
london_venues_stats.head()

In [55]:
london_eating_venues = pd.merge(london_eats, london_venues_stats, how='left', on='Venue ID')

In [60]:
london_eating_venues.head()

,index,LSOA name,LSOA Latitude,LSOA Longitude,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Venue name,Tip Count,Rating,Price tier
0,9,City of London 001A,51.51801,-0.09677,4ad7a8ddf964a520650d21e3,Dose Espresso,51.519553,-0.099406,Coffee Shop,Dose Espresso,69.0,8.4,2.0
1,11,City of London 001A,51.51801,-0.09677,5384e5ed498e11317d174b6d,Ask For Janice,51.519128,-0.100374,Modern European Restaurant,Ask For Janice,65.0,8.4,2.0
2,12,City of London 001A,51.51801,-0.09677,4ada5cf0f964a520e32121e3,The Old Red Cow,51.519607,-0.099157,Beer Bar,The Old Red Cow,59.0,8.2,2.0
3,13,City of London 001A,51.51801,-0.09677,4dff32bae4cdf7246077a9aa,Pilpel,51.515195,-0.098462,Falafel Restaurant,Pilpel,36.0,9.1,2.0
4,18,City of London 001A,51.51801,-0.09677,4ac518d6f964a5201ea820e3,Club Gascon,51.518541,-0.100462,French Restaurant,Club Gascon,21.0,8.0,3.0


In [59]:
london_eating_venues.shape

(1018, 13)

dropping columns that are no longer needed

In [61]:
london_eating_venues_clean = london_eating_venues.drop(['index','LSOA Latitude', 'LSOA Longitude', 'Venue ID', 'Venue Latitude', 'Venue Longitude', 'Venue Name', 'Venue name'], axis=1)

In [62]:
london_eating_venues_clean.head()

,LSOA name,Venue Category,Tip Count,Rating,Price tier
0,City of London 001A,Coffee Shop,69.0,8.4,2.0
1,City of London 001A,Modern European Restaurant,65.0,8.4,2.0
2,City of London 001A,Beer Bar,59.0,8.2,2.0
3,City of London 001A,Falafel Restaurant,36.0,9.1,2.0
4,City of London 001A,French Restaurant,21.0,8.0,3.0


In [63]:
project.save_data(file_name = "london_eating_venues_stats.csv", data = london_eating_venues_clean.to_csv(index=False),overwrite=True)

{'file_name': 'london_eating_venues_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstoneibmdatascience-donotdelete-pr-ur8dfm3zrk3isl',
 'asset_id': '388988e4-3c64-45a5-8595-a913d31f675e'}

In [64]:
london_eating_venues_clean.describe()

,Tip Count,Rating,Price tier
count,1012.000000,1002.000000,982.000000
mean,36.842885,8.130040,1.690428
std,60.689184,0.677146,0.719532
min,0.000000,5.900000,1.000000
25%,6.000000,7.800000,1.000000
50%,17.000000,8.200000,2.000000
75%,42.000000,8.700000,2.000000
max,517.000000,9.200000,4.000000


In [65]:
london_eating_venues_clean.dtypes

LSOA name          object
Venue Category     object
Tip Count         float64
Rating            float64
Price tier        float64
dtype: object

In [68]:
london_eating_venues_clean['LSOA-Venue type'] = london_eating_venues_clean[['LSOA name', 'Venue Category']].apply(lambda x: '-'.join(x), axis=1)

In [73]:
london_eating_venues_clean = london_eating_venues_clean.drop(['LSOA name', 'Venue Category'], axis=1)

In [76]:
cols = list(london_eating_venues_clean.columns)
cols = [cols[-1]] + cols[:-1]
london_eating_venues_clean = london_eating_venues_clean[cols]

In [77]:
london_eating_venues_clean.head()

,LSOA-Venue type,Tip Count,Rating,Price tier
0,City of London 001A-Coffee Shop,69.0,8.4,2.0
1,City of London 001A-Modern European Restaurant,65.0,8.4,2.0
2,City of London 001A-Beer Bar,59.0,8.2,2.0
3,City of London 001A-Falafel Restaurant,36.0,9.1,2.0
4,City of London 001A-French Restaurant,21.0,8.0,3.0


In [80]:
london_venues_grouped = london_eating_venues_clean.groupby(['LSOA-Venue type']).mean().reset_index()
london_venues_grouped.head()

,LSOA-Venue type,Tip Count,Rating,Price tier
0,City of London 001A-Bar,68.0,9.10,3.0
1,City of London 001A-Beer Bar,59.0,8.20,2.0
2,City of London 001A-Coffee Shop,35.0,8.15,1.5
3,City of London 001A-English Restaurant,151.0,8.50,3.0
4,City of London 001A-Falafel Restaurant,36.0,9.10,2.0


In [81]:
interim = london_venues_grouped["LSOA-Venue type"].str.split("-", n = 1, expand = True)

In [83]:
london_venues_grouped['LSOA'] = interim[0]
london_venues_grouped['Venue type'] = interim[1]
london_venues_grouped = london_venues_grouped.drop(['LSOA-Venue type'], axis=1)

In [85]:
cols = list(london_venues_grouped.columns)
cols = [cols[-1]] + cols[:-1]
london_venues_grouped = london_venues_grouped[cols]

london_venues_grouped.head()

,Venue type,Tip Count,Rating,Price tier,LSOA
0,Bar,68.0,9.10,3.0,City of London 001A
1,Beer Bar,59.0,8.20,2.0,City of London 001A
2,Coffee Shop,35.0,8.15,1.5,City of London 001A
3,English Restaurant,151.0,8.50,3.0,City of London 001A
4,Falafel Restaurant,36.0,9.10,2.0,City of London 001A


In [86]:
cols = list(london_venues_grouped.columns)
cols = [cols[-1]] + cols[:-1]
london_venues_grouped = london_venues_grouped[cols]

In [93]:
london_venues_grouped.head()

,LSOA,Venue type,Tip Count,Rating,Price tier
0,City of London 001A,Bar,68.0,9.10,3.0
1,City of London 001A,Beer Bar,59.0,8.20,2.0
2,City of London 001A,Coffee Shop,35.0,8.15,1.5
3,City of London 001A,English Restaurant,151.0,8.50,3.0
4,City of London 001A,Falafel Restaurant,36.0,9.10,2.0


In [88]:
london_venues_grouped.shape

(229, 5)

#### Merge with LSOA stats

In [114]:
#lsoa_london_coordinates_clean = lsoa_london_coordinates_clean.rename(columns = {'Names':'LSOA'})

london_merged = pd.merge(london_venues_grouped, lsoa_london_coordinates_clean, how='left', on='LSOA')

london_merged = london_merged.drop(['Lower Super Output Area', 'Latitude', 'Longitude'], axis =1)

london_merged.head()

,LSOA,Venue type,Tip Count,Rating,Price tier,Population Density;Persons per hectare;2013,Households;All households;2011,House Prices;Median Price (£);2009,House Prices;Median Price (£);2014,House Prices;Sales;2009,House Prices;Sales;2014,Economic Activity;Economically active: Total;2011,Economic Activity;Unemployment Rate;2011,"Household Income, 2011/12;Mean Annual Household Income estimate (£)",House Prices;Median Price (£);CAGR,House Prices;Sales;CAGR
0,City of London 001A,Bar,68.0,9.10,3.0,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297
1,City of London 001A,Beer Bar,59.0,8.20,2.0,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297
2,City of London 001A,Coffee Shop,35.0,8.15,1.5,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297
3,City of London 001A,English Restaurant,151.0,8.50,3.0,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297
4,City of London 001A,Falafel Restaurant,36.0,9.10,2.0,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297


### 3.5 Applying clustering analysis

In [115]:
london_merged.head()

,LSOA,Venue type,Tip Count,Rating,Price tier,Population Density;Persons per hectare;2013,Households;All households;2011,House Prices;Median Price (£);2009,House Prices;Median Price (£);2014,House Prices;Sales;2009,House Prices;Sales;2014,Economic Activity;Economically active: Total;2011,Economic Activity;Unemployment Rate;2011,"Household Income, 2011/12;Mean Annual Household Income estimate (£)",House Prices;Median Price (£);CAGR,House Prices;Sales;CAGR
0,City of London 001A,Bar,68.0,9.10,3.0,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297
1,City of London 001A,Beer Bar,59.0,8.20,2.0,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297
2,City of London 001A,Coffee Shop,35.0,8.15,1.5,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297
3,City of London 001A,English Restaurant,151.0,8.50,3.0,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297
4,City of London 001A,Falafel Restaurant,36.0,9.10,2.0,114.0,876.0,480000.0,720000,41.0,67.0,964.0,3.5,74846.0,0.069913,0.085297


In [116]:
london_merged.dtypes

LSOA                                                                    object
Venue type                                                              object
Tip Count                                                              float64
Rating                                                                 float64
Price tier                                                             float64
Population Density;Persons per hectare;2013                            float64
Households;All households;2011                                         float64
House Prices;Median Price (£);2009                                     float64
House Prices;Median Price (£);2014                                       int64
House Prices;Sales;2009                                                float64
House Prices;Sales;2014                                                float64
Economic Activity;Economically active: Total;2011                      float64
Economic Activity;Unemployment Rate;2011            

In [120]:
london_merged.isnull().sum()

LSOA                                                                   0
Venue type                                                             0
Tip Count                                                              0
Rating                                                                 0
Price tier                                                             0
Population Density;Persons per hectare;2013                            0
Households;All households;2011                                         0
House Prices;Median Price (£);2009                                     0
House Prices;Median Price (£);2014                                     0
House Prices;Sales;2009                                                0
House Prices;Sales;2014                                                0
Economic Activity;Economically active: Total;2011                      0
Economic Activity;Unemployment Rate;2011                               0
Household Income, 2011/12;Mean Annual Household Inc

In [119]:
london_merged = london_merged.dropna()

In [124]:

float_col = london_merged.select_dtypes(include=['float64']) 
for col in float_col.columns.values:
     london_merged[col] = london_merged[col].astype('int64')
        
        
london_merged.dtypes

LSOA                                                                   object
Venue type                                                             object
Tip Count                                                               int64
Rating                                                                  int64
Price tier                                                              int64
Population Density;Persons per hectare;2013                             int64
Households;All households;2011                                          int64
House Prices;Median Price (£);2009                                      int64
House Prices;Median Price (£);2014                                      int64
House Prices;Sales;2009                                                 int64
House Prices;Sales;2014                                                 int64
Economic Activity;Economically active: Total;2011                       int64
Economic Activity;Unemployment Rate;2011                        

In [132]:
project.save_data(file_name = "london_merged.csv", data = london_merged.to_csv(index=False),overwrite=True)

{'file_name': 'london_merged.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstoneibmdatascience-donotdelete-pr-ur8dfm3zrk3isl',
 'asset_id': '6a74542d-e2d3-48ac-8407-4ee484ad8acd'}

In [133]:
kclusters = 4

london_clustering = london_merged.drop(['LSOA','Venue type'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_clustering)

kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [134]:
# add clustering labels
london_merged.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

## 4. Results

We have done modelling part and clustered all venue types within particular LSOA by appropriate cluster. Here are the results

In [135]:
london_merged.head(10)

,Cluster Labels,LSOA,Venue type,Tip Count,Rating,Price tier,Population Density;Persons per hectare;2013,Households;All households;2011,House Prices;Median Price (£);2009,House Prices;Median Price (£);2014,House Prices;Sales;2009,House Prices;Sales;2014,Economic Activity;Economically active: Total;2011,Economic Activity;Unemployment Rate;2011,"Household Income, 2011/12;Mean Annual Household Income estimate (£)",House Prices;Median Price (£);CAGR,House Prices;Sales;CAGR
219,1,Kensington and Chelsea 006A,Bakery,81,8,1,110,703,790000,1175000,16,17,767,5,69961,0,0
220,1,Kensington and Chelsea 006A,Bar,3,8,2,110,703,790000,1175000,16,17,767,5,69961,0,0
221,1,Kensington and Chelsea 006A,Breakfast Spot,30,8,1,110,703,790000,1175000,16,17,767,5,69961,0,0
222,1,Kensington and Chelsea 006A,Burger Joint,59,8,2,110,703,790000,1175000,16,17,767,5,69961,0,0
223,1,Kensington and Chelsea 006A,Coffee Shop,11,9,1,110,703,790000,1175000,16,17,767,5,69961,0,0
224,1,Kensington and Chelsea 006A,Diner,181,8,3,110,703,790000,1175000,16,17,767,5,69961,0,0
225,1,Kensington and Chelsea 006A,Gastropub,58,8,4,110,703,790000,1175000,16,17,767,5,69961,0,0
226,1,Kensington and Chelsea 006A,Italian Restaurant,42,8,2,110,703,790000,1175000,16,17,767,5,69961,0,0
227,1,Kensington and Chelsea 006A,Pub,48,7,1,110,703,790000,1175000,16,17,767,5,69961,0,0
228,1,Kensington and Chelsea 006A,Restaurant,54,8,2,110,703,790000,1175000,16,17,767,5,69961,0,0


In [136]:
london_merged.tail(10)

,Cluster Labels,LSOA,Venue type,Tip Count,Rating,Price tier,Population Density;Persons per hectare;2013,Households;All households;2011,House Prices;Median Price (£);2009,House Prices;Median Price (£);2014,House Prices;Sales;2009,House Prices;Sales;2014,Economic Activity;Economically active: Total;2011,Economic Activity;Unemployment Rate;2011,"Household Income, 2011/12;Mean Annual Household Income estimate (£)",House Prices;Median Price (£);CAGR,House Prices;Sales;CAGR
219,1,Kensington and Chelsea 006A,Bakery,81,8,1,110,703,790000,1175000,16,17,767,5,69961,0,0
220,1,Kensington and Chelsea 006A,Bar,3,8,2,110,703,790000,1175000,16,17,767,5,69961,0,0
221,1,Kensington and Chelsea 006A,Breakfast Spot,30,8,1,110,703,790000,1175000,16,17,767,5,69961,0,0
222,1,Kensington and Chelsea 006A,Burger Joint,59,8,2,110,703,790000,1175000,16,17,767,5,69961,0,0
223,1,Kensington and Chelsea 006A,Coffee Shop,11,9,1,110,703,790000,1175000,16,17,767,5,69961,0,0
224,1,Kensington and Chelsea 006A,Diner,181,8,3,110,703,790000,1175000,16,17,767,5,69961,0,0
225,1,Kensington and Chelsea 006A,Gastropub,58,8,4,110,703,790000,1175000,16,17,767,5,69961,0,0
226,1,Kensington and Chelsea 006A,Italian Restaurant,42,8,2,110,703,790000,1175000,16,17,767,5,69961,0,0
227,1,Kensington and Chelsea 006A,Pub,48,7,1,110,703,790000,1175000,16,17,767,5,69961,0,0
228,1,Kensington and Chelsea 006A,Restaurant,54,8,2,110,703,790000,1175000,16,17,767,5,69961,0,0


## 5. Discussion of the results

Even though clustering was appropriate, algorithm could be significanlty improved if additional data points about particular venue are added to the dataset. 

## 6. Conclusions

Clustering analysis showed us which venue types & LSOA could be grouped together by various parameters. 